In [1269]:
import pandas as pd
import numpy as np
from datetime import date
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [1270]:
# Input the data
data = pd.read_csv("C:\\Users\\Chase\\Downloads\\Preppin' Summer 2022 - Store Mortgages.csv")
data

,Store,Capital Repayment Remaining,Monthly Payment,% of Monthly Repayment going to Capital
0,Lewisham,250000,1500,60
1,Wimbledon,350000,1900,55


In [1271]:
# Create a field for today (8/10/2022)
data['Today'] = '07/11/2022'
data

,Store,Capital Repayment Remaining,Monthly Payment,% of Monthly Repayment going to Capital,Today
0,Lewisham,250000,1500,60,07/11/2022
1,Wimbledon,350000,1900,55,07/11/2022


In [1272]:
# Create a data field to show how much capital is paid off each month
data['$ of Monthly Repayment going to Capital'] = data['Monthly Payment'] * (data["% of Monthly Repayment going to Capital"] /100)
data

,Store,Capital Repayment Remaining,Monthly Payment,% of Monthly Repayment going to Capital,Today,$ of Monthly Repayment going to Capital
0,Lewisham,250000,1500,60,07/11/2022,900.0
1,Wimbledon,350000,1900,55,07/11/2022,1045.0


In [1273]:
# Create a data field to show how many months are needed to pay off the entire debt (whole months only)
data['# of Months to Repayment'] = round(data['Capital Repayment Remaining'] / data['$ of Monthly Repayment going to Capital'], 0)
data['# of Months to Repayment'] = data['# of Months to Repayment'].astype(int)
data

,Store,Capital Repayment Remaining,Monthly Payment,% of Monthly Repayment going to Capital,Today,$ of Monthly Repayment going to Capital,# of Months to Repayment
0,Lewisham,250000,1500,60,07/11/2022,900.0,278
1,Wimbledon,350000,1900,55,07/11/2022,1045.0,335


In [1274]:
# Create a field when the mortgages will be paid off (Assuming a payment is to be made in August 2022)
def add_months(start_date, delta_period):
    end_date = start_date + relativedelta(months=delta_period)
    return end_date
tqdm.pandas()
data['Today'] = pd.to_datetime(data['Today'])
data['Capital Repayment Date'] = data.progress_apply(lambda row: add_months(row["Today"], row['# of Months to Repayment']), axis= 1)
data['Capital Repayment Date'] = data['Capital Repayment Date'].dt.strftime('%m/%d/%Y')
data

100%|██████████| 2/2 [00:00<00:00, 1998.24it/s]


,Store,Capital Repayment Remaining,Monthly Payment,% of Monthly Repayment going to Capital,Today,$ of Monthly Repayment going to Capital,# of Months to Repayment,Capital Repayment Date
0,Lewisham,250000,1500,60,2022-07-11,900.0,278,09/11/2045
1,Wimbledon,350000,1900,55,2022-07-11,1045.0,335,06/11/2050


In [1275]:
# Create a row per month between now and when the mortgage is paid off showing:
    # 1. How much is still to be paid off for that mortgage? Call this field 'Remaining Capital to Repay' 
    # 2. How much is still to be paid off for all mortgages? Call this field 'Capital Outstanding Total'

# Set initial total capital balance
data['Capital Outstanding Total'] = data['Capital Repayment Remaining'].sum()
# Calculate total monthly capital payments
data['Monthly Payment'] = data['$ of Monthly Repayment going to Capital'].sum()
# Create dataframe for the Lewisham store
data2 = data[data['Store'] == 'Lewisham']
# Create column specifying the individual monthly contributions for each store
data2['Lewisham Monthly Payment'] = np.where(data2['Store'] == 'Lewisham', 900.0, 1045.0)
# Convert Repayment date column to datetime
data2['Capital Repayment Date'] = pd.to_datetime(data2['Capital Repayment Date'])
# Scaffold the dates
data2['Monthly Payment Date'] = [pd.date_range(x, y, freq='M') + pd.DateOffset(days=11) for x, y in zip(data2['Today'], data2['Capital Repayment Date'])]
# Show the date scaffold
data2 = data2.explode('Monthly Payment Date')
# Create temp dataframe for remaining capital
s = {'Monthly Payment': 600000 - data2['Monthly Payment'].cumsum()}
# Calculate the reverse cumulative sum
data2['Capital Outstanding Total'] = s['Monthly Payment']
# Create temp dataframe for remaining individual capital
d = {'Store': data2['Store'], 'Capital Outstanding Data': 250000 - data2['Lewisham Monthly Payment'].cumsum()}
# Calculate the reversecumulative sum for capital remaining
data2['Remaining Capital to Repay'] = d['Capital Outstanding Data']
# Reorganize columns
data2 = data2[['Monthly Payment Date', 'Store', 'Capital Outstanding Total', 'Remaining Capital to Repay']]


# Rinse and Repeat the above process with the Wimbledon store
data3 = data[data['Store'] == 'Wimbledon']
data3['Wimbledon Monthly Payment'] = np.where(data3['Store'] == 'Wimbledon', 1045.0, 900.0)
data3['Capital Repayment Date'] = pd.to_datetime(data3['Capital Repayment Date'])
data3['Monthly Payment Date'] = [pd.date_range(x, y, freq='M') + pd.DateOffset(days=11) for x, y in zip(data3['Today'], data3['Capital Repayment Date'])] 
data3['Capital Outstanding Total'] = 600000 - data3['Monthly Payment'].cumsum()
data3 = data3.explode('Monthly Payment Date')
mask = data3[data3['Monthly Payment Date'] > '11/11/2045']
d = {'Store': data3['Store'], 'Capital Outstanding Data': 350000 - data3['Wimbledon Monthly Payment'].cumsum()}
data3['Remaining Capital to Repay'] = d['Capital Outstanding Data']
# Only difference is that when the loan is paid off for Lewisham, the total capital remaining equals the capital remaining for Wimbledon
s = {'Monthly Payment': np.where(data3['Monthly Payment Date'] < '11/11/2045', 600000 - data3['Monthly Payment'].cumsum(), data3['Remaining Capital to Repay'])}
data3['Capital Outstanding Total'] = s['Monthly Payment']
data3 = data3[['Monthly Payment Date', 'Store', 'Capital Outstanding Total', 'Remaining Capital to Repay']]
final = pd.concat([data2, data3])
final['Monthly Payment Date'] = final['Monthly Payment Date'].dt.strftime('%m/%d/%Y')
final

,Monthly Payment Date,Store,Capital Outstanding Total,Remaining Capital to Repay
0,08/11/2022,Lewisham,598055.0,249100.0
0,09/11/2022,Lewisham,596110.0,248200.0
0,10/11/2022,Lewisham,594165.0,247300.0
0,11/11/2022,Lewisham,592220.0,246400.0
0,12/11/2022,Lewisham,590275.0,245500.0
...,...,...,...,...
1,02/11/2050,Wimbledon,4105.0,4105.0
1,03/11/2050,Wimbledon,3060.0,3060.0
1,04/11/2050,Wimbledon,2015.0,2015.0
1,05/11/2050,Wimbledon,970.0,970.0
